In [ ]:
# import lib
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchsummary
import torchvision.transforms as transforms
import time
from torch.autograd import Function
!/opt/bin/nvidia-smi

Wed May 12 04:31:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# load data

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

# Hyper-parameters
input_size = 784
hidden_size = [2048, 2048]
num_classes = 10
batch_size = 32
#learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# define normal_sign
class NormalSign(Function):
    @staticmethod
    def forward(ctx, inputs):
        ctx.save_for_backward(inputs)
        outputs = torch.ones_like(inputs)
        outputs[inputs<0] = -1.
        return outputs
    @staticmethod
    def backward(ctx, dy):
        dx = dy.clone()
        return dx

In [ ]:
class BinaryDenseNet(nn.Module):
    def __init__(self, input_features, output_features):
        super(BinaryDenseNet, self).__init__()
        self.input_features = input_features
        self.ouput_features = output_features
        self.W_LR_scale = torch.tensor(1./np.sqrt(1.5/ (input_features + output_features)))
        self.weights = nn.Parameter(2.*torch.rand(input_features, output_features)-1.)
        self.sign = NormalSign.apply
        
    def forward(self, inputs):
        outputs = self.sign(inputs @ self.sign(self.weights))
        #print(self.weights)
       # print('size of input', inputs.size())
        #print('size of self.weights', self.weights.size())
        #print(inputs @ self.weights)
        #print('bdn', torch.norm(outputs[1]-outputs[2]))
        return outputs,self.weights

In [ ]:
# define model 2 layers
# Fully connected neural network with one hidden layer
class BNN(nn.Module):
    def __init__(self, input_size=784, hidden_size=(128,128), num_classes=10, batch_size=64):
        super(BNN, self).__init__()
        self.fc1 = BinaryDenseNet(input_size, hidden_size[0])
        self.fc2 = BinaryDenseNet(hidden_size[0], hidden_size[1])
        self.w3 = nn.Parameter(2.*torch.rand(hidden_size[1], num_classes)-1.)
        self.sign = NormalSign.apply
    def forward(self, x):
        x1,w1 = self.fc1(self.sign(2*x-1))
        x2,w2 = self.fc2(self.sign(2*x1-1))
        out = x2 @ self.sign(self.w3)
        return out


model = BNN(hidden_size=(32,32)).to(device)
#torchsummary.summary(model, torch.Size([784]), batch_size)

In [ ]:
num_epochs = 100
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# adjust learining rate. We adopt the exponential decaying rule
lr_start = 0.001
lr_fin = 0.000003
lr_decay = (lr_fin/lr_start)**(1./num_epochs)

optimizer = torch.optim.Adam(model.parameters(), lr=lr_start)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=lr_decay)

# Train the model
total_step = len(train_loader)

special_layers_dict = torch.nn.ModuleDict({})
for name,m in model.named_modules():
    if isinstance(m, BinaryDenseNet):
        special_layers_dict.update({name:m})
        
print(special_layers_dict)
for epoch in range(120):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        binary_module = torch.nn.ModuleDict({})
        
        
        # record old parameter
        old_parameters = {}
        for name,value in special_layers_dict.items():
            for para in value.parameters():
                old_parameters[name] = para.clone().detach()
        
        # update parameters
        optimizer.step()
        # scale the weights learning rates respectively with the weights initialization coefficeitns and clip()
        with torch.no_grad():
            for name,value in special_layers_dict.items():
                for para in value.parameters():
                    para += (value.W_LR_scale - 1.) * (para - old_parameters[name])
                    para.clamp_(min=-1., max=1.)

        if (i) % 500 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, learning rate = {},'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),lr_scheduler.get_lr()))
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    images = images.reshape(-1, 28 * 28).to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
    # after every epoch, update learning rate
    lr_scheduler.step()


            

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
#torch.save(model.state_dict(), 'DNN-model.ckpt')

ModuleDict(
  (fc1): BinaryDenseNet()
  (fc2): BinaryDenseNet()
)


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:449: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Epoch [1/100], Step [1/1875], Loss: 9.3679, learning rate = [0.001],
Accuracy of the network on the 10000 test images: 9.44 %
Epoch [1/100], Step [501/1875], Loss: 1.6627, learning rate = [0.001],
Accuracy of the network on the 10000 test images: 62.11 %
Epoch [1/100], Step [1001/1875], Loss: 2.0279, learning rate = [0.001],
Accuracy of the network on the 10000 test images: 72.69 %
Epoch [1/100], Step [1501/1875], Loss: 0.8374, learning rate = [0.001],
Accuracy of the network on the 10000 test images: 72.48 %
Epoch [2/100], Step [1/1875], Loss: 2.1416, learning rate = [0.000890312406057461],
Accuracy of the network on the 10000 test images: 71.85 %
Epoch [2/100], Step [501/1875], Loss: 2.1079, learning rate = [0.000890312406057461],
Accuracy of the network on the 10000 test images: 72.55 %
Epoch [2/100], Step [1001/1875], Loss: 2.6409, learning rate = [0.000890312406057461],
Accuracy of the network on the 10000 test images: 72.6 %
Epoch [2/100], Step [1501/1875], Loss: 0.7977, learning

In [ ]:
model_dict = model.named_parameters
parameters = list(model.parameters())[-33:]
##np.savetxt("filename.txt",a)
np.save("w2.npy",parameters[0].cpu().detach().numpy())
np.save("w0.npy",parameters[1].cpu().detach().numpy())
np.save("w1.npy",parameters[2].cpu().detach().numpy())